In [2]:
import glob
import random
import h5py
import numpy as np

Find data files recursively from root folder.

In [3]:
data_path = "D:/Coding/Thesis/Data/STFT Output/**/*.h5"
data_files = glob.glob(data_path, recursive=True)

In [4]:
config = {}

config['EEG_window_length_in_samples'] = 30000

The ParticipantData class contains and acts on participant data. It knows about the structure of the h5 files and can return random windows of EEG activity.

In [47]:
class ParticipantData:
    def __init__(self, data_files):
        self.data_file_paths = data_files
        
        self.open_h5_files()
        
        self._generate_group_idx()
    
    
    def _generate_group_idx(self):
        self.sessions = [list(participant.keys()) for participant in self.data_files_open]

        #windows = [list(participant[session].keys()) for participant in data_files_open for session in list(participant.keys())]
        self.windows = []
        for participant in self.data_files_open:
            for session in list(participant.keys()):
                self.windows.append(list(participant[session].keys()))
    
    
    def open_h5_files(self):
        self.data_files_open = [h5py.File(f, 'r') for f in self.data_file_paths]
    
    
    def close_h5_files(self):
        for f in self.data_files_open:
            f.close()
        
        self.data_files_open = []
    
    
    def get_random_EEG_window(self, window_size):
        ppt = random.choice(self.data_files_open)        
        session = random.choice(list(ppt.keys()))
        activity_window = random.choice(list(ppt[session].keys()))
        
        window_start_idx = random.randrange(ppt[session][activity_window]['stft'].shape[0] - window_size)
        
        window_idx = range(window_start_idx, window_start_idx + window_size)
        

        input_data = np.array(ppt[session][activity_window]['stft'][window_idx, :, :][0])
        
        output_data = self.get_taps_in_window(ppt[session][activity_window]['taps'], window_idx[0], window_idx[-1])
                
        return(input_data, output_data)
    
    
    def get_taps_in_window(self, taps, window_start, window_end):
        _taps = np.array(taps)
        
        self.get_delta_taps()
        
        self.get_n_future_taps()
        
        result = []
        
        return(result)
    
    def get_delta_taps(self):
        pass
    
    def get_n_future_taps(self):
        pass

In [48]:
ppt_data = ParticipantData(data_files)

In [49]:
ppt_data.get_random_EEG_window(config['EEG_window_length_in_samples'])

(array([[12.13350964,  7.41144276,  9.55447197, 10.58475971, 10.4837513 ,
          8.28670597,  9.07802105,  8.24339294,  7.65415144,  8.15209675,
          8.81490993,  7.20322561,  7.47662544,  7.91811657,  5.72772503,
          7.08757448,  7.71772289,  6.81636095,  6.97858763,  4.58052301,
          6.42834949,  8.07438469,  7.25997162,  7.00556755,  7.32896805,
          6.46357298,  7.35826492,  7.49879503,  3.22935486,  6.00881433,
          6.71954584,  6.04467201]]), [])

Assign real and imaginary part of stft output separately to new numpy array.

In [45]:
f = h5py.File(data_files[0], 'r')

list(f['12_02_11_04_19']['window_1'].keys())

print(f['12_02_11_04_19']['window_1']['taps'][:, -1][0])
print(f['12_02_11_04_19']['window_1']['stft'].shape[0])

3112552
3142553


Close h5 files again.

In [50]:
ppt_data.close_h5_files()